<a href="https://colab.research.google.com/github/fengxhao/colab/blob/main/Recursive_Retriever_Advanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index sentence-transformers rank-bm25
!pip install -q llama-index-llms-groq llama-index-embeddings-huggingface llama-index-retrievers-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
# apply nested async to run in a notebook
nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.groq import Groq
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core import Settings
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.node_parser import SimpleNodeParser,SentenceWindowNodeParser
from llama_index.core.schema import IndexNode
from llama_index.core import VectorStoreIndex
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import LongContextReorder
from llama_index.core.response.notebook_utils import display_response
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core import SummaryIndex
from llama_index.core.extractors import SummaryExtractor
from llama_index.core import get_response_synthesizer
import copy
from typing import List
from llama_index.readers.file import PyMuPDFReader
import re

In [ ]:
import os
os.environ["groq_api_key"] = "Your groq api key" # 免费注册获取 https://console.groq.com/keys

## LlamaIndex Recursive Retriever

通常在构建 RAG 流程时，不同的知识库文档，可能需要从许多检索参数/策略中选择合适的处理方式，涉及分块大小，是否生成摘要，是否假设答案/提问，是否rerank，是否处理LongContext Middle，是否使用融合算法...

希望在RAG流程中自定义执行更多的策略，可以试试 **Recursive Retriever**

主要思路：

不仅 Retrieval 直接最相关的节点，而且 Retrieval 到的节点与其他Retriever/QueryEngine/IndexNode指向的关系会去进一步执行。例如，文档有文字和图表，文字分块处理正常执行，同时构建 Index 节点可以表示结构化表的简明摘要，并索引到该结构化表上的 SQL/Pandas QueryEngine去执行获取明确结果。


### 本示例综合 Recursive Retriever 的索引能力尝试自定义 RAG 流程

测试文档分别是两位名人的Wiki
1. [Michael Jordan](https://en.wikipedia.org/wiki/Michael_Jordan)
2. [Elon Musk](https://en.wikipedia.org/wiki/Elon_Musk)

希望的 RAG 流程是：
* 生成两篇 Wiki 的摘要作为 Top IndexNode，来索引对应的 Wiki 生成的 Documents 各自的 Retriever， 通过RecursiveRetriever 来递归执行。
* Michael Jordan Wiki 的Retriever构建：

  1. SentenceSplitter 将 Document 分成 512 的Chunk作为 BaseNodes。
  2. 使用 SentenceWindowNodeParser 将 BaseNode 进一步处理成带前后句 Window 的单句 SubNode。SubNode 的 Index Id指向 BaseNode。
  3. 所有的 Nodes Flat 后进入 VectorStoreIndex 中，并创建Retriever。
  4. 使用 RecursiveRetriever 来 Wrap Vector Retriever，这样在 retrieval到subNode会索引到BaseNode，这样传递给LLM的上下文更丰富具体。

* Elon Musk Wiki 的Retriever构建：
  
  1. SentenceSplitter 将 Document 分成 512 的Chunk作为 BaseNodes。
  2. 对 BaseNodes 进行 Tree Summarize，并存入metaData中，同时将 Summary创建IndexNode索引到 BaseNode。
  3. 所有的 Nodes Flat 后进入 VectorStoreIndex 中，并创建Retriever。
  4. 使用 BM25Retriever 补充对关键词的检索能力，并用 QueryFusionRetriever 倒数重排 Vector 和 BM25的 Node结果。


**这个执行流程有点冗余，纯属娱乐。：）**

---

欢迎关注我的[小红书：AGI悟道](https://www.xiaohongshu.com/user/profile/64cdc8b0000000000b006bff?xhsshare=WeixinSession&appuid=64cdc8b0000000000b006bff&apptime=1715318887&wechatWid=5cd9ba53a281612dd8c33e00d51b9aad&wechatOrigin=menu)，发现更多有趣的AI Demo～

### LLM
* 主要的 LLM 使用 Groq 提供的 Llama3-70b，做摘要的LLM 使用llama3-8b-8192（Token rate limit比较大）
* Embedding 使用 BGE-Large

In [ ]:
  embed_model = resolve_embed_model("local:BAAI/bge-large-en-v1.5")
  llm = Groq(model="llama3-70b-8192", api_key=os.environ["groq_api_key"])
  llm_summarize = Groq(model="llama3-8b-8192", api_key=os.environ["groq_api_key"])

  Settings.llm = llm
  Settings.embed_model = embed_model

### 测试文档
* Michael Jordan Wiki: https://en.wikipedia.org/wiki/Michael_Jordan
* Elon_Musk Wiki: https://en.wikipedia.org/wiki/Elon_Musk

In [ ]:
from pathlib import Path

!mkdir data
wiki_titles = ["Michael Jordan", "Elon Musk"]

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

### 加载文档生成两篇 Wiki 的 Doucments

In [ ]:
docs_dict = {}
for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()[0] # 太多太消耗资源，这里只取第一部分Documents测试

    docs_dict[wiki_title] = doc

documents_jordan = [docs_dict["Michael Jordan"]]
documents_musk = [docs_dict["Elon Musk"]]

print("\n===========Michael Jordan Wiki=============")
print(documents_jordan[0].text)
print("===========================")
print(documents_jordan[0].metadata)

print("\n===========Elon Musk Wiki==================")
print(documents_musk[0].text)
print("===========================")
print(documents_musk[0].metadata)

## Michael Jordan Wiki 处理流程

### 分块生成 BaseNode 并指定 Node Id（不指定会随机生成）

In [ ]:
node_parser = SentenceSplitter(chunk_size=512)

base_nodes_jordan = node_parser.get_nodes_from_documents(documents_jordan)
for idx, node in enumerate(base_nodes_jordan):
    node.id_ = f"jordan_node-{idx}"


### 用 SentenceWindowNodeParser 生成带前后句 Window 的 Sentence Node

In [ ]:
sentence_window_node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text")

### 将BaseNode 用 SentenceWindowNodeParser 拆成 SubNode，并索引向 BaseNode

In [ ]:
all_nodes_jordan = []
for base_node in base_nodes_jordan:
    sub_nodes = sentence_window_node_parser.get_nodes_from_documents([base_node])
    sub_inodes = [
        IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
    ]
    all_nodes_jordan.extend(sub_inodes)
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes_jordan.append(original_node)

all_nodes_dict_jordan = {n.node_id: n for n in all_nodes_jordan}


print(len(all_nodes_jordan))
print(all_nodes_jordan[10].relationships)


746
{<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='jordan_node-0', node_type=<ObjectType.TEXT: '1'>, metadata={'file_path': 'data/Michael Jordan.txt', 'file_name': 'Michael Jordan.txt', 'file_type': 'text/plain', 'file_size': 66023, 'creation_date': '2024-05-10', 'last_modified_date': '2024-05-10'}, hash='1bc45f6d4aa63317f8f48f096f0ad2f89bfc1e8eddfe9f9c5bf0668584523c1b'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='9a9ebf48-3290-499e-baf7-77cdbddee34e', node_type=<ObjectType.TEXT: '1'>, metadata={'window': 'Jordan joined the Bulls in 1984 as the third overall draft pick and quickly emerged as a league star, entertaining crowds with his prolific scoring while gaining a reputation as one of the best defensive players.  His leaping ability, demonstrated by performing slam dunks from the free-throw line in Slam Dunk Contests, earned him the nicknames "Air Jordan" and "His Airness".  Jordan won his first NBA title with the Bulls in 1991 and followed that achievemen

### 通过 VectorStoreIndex 将 Michael Jordan Wiki 的所有节点入向量数据库

In [ ]:
vector_index_chunk_jordan = VectorStoreIndex(all_nodes_jordan)

### 使用 RecursiveRetriever 来做递归检索，当问题相似性召回子节点，会索引到父节点获取更丰富的内容

In [ ]:
vector_retriever_chunk_jordan = vector_index_chunk_jordan.as_retriever(similarity_top_k=5)

recursive_retriever_jordan = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk_jordan},
    node_dict=all_nodes_dict_jordan,
    verbose=True,
)


### 测试 Michael Jordan Wiki Retriever

In [ ]:
nodes = recursive_retriever_jordan.retrieve(
    "Tell me about a celebrity from the United States"
)
for node in nodes:
    print(node.node.get_content())


Retrieving with query id None: Tell me about a celebrity from the United States
Retrieved node with id, entering: jordan_node-0
Retrieving with query id jordan_node-0: Tell me about a celebrity from the United States
Retrieved node with id, entering: jordan_node-23
Retrieving with query id jordan_node-23: Tell me about a celebrity from the United States
Retrieved node with id, entering: jordan_node-39
Retrieving with query id jordan_node-39: Tell me about a celebrity from the United States
Retrieved node with id, entering: jordan_node-40
Retrieving with query id jordan_node-40: Tell me about a celebrity from the United States
Retrieved node with id, entering: jordan_node-42
Retrieving with query id jordan_node-42: Tell me about a celebrity from the United States
Michael Jeffrey Jordan (born February 17, 1963), also known by his initials MJ, is an American businessman and former professional basketball player. He played fifteen seasons in the National Basketball Association (NBA) betwee

## Elon_Musk Wiki 处理流程

### 分块生成 BaseNode 并指定 Node Id（不指定会随机生成）

In [ ]:
base_nodes_musk = node_parser.get_nodes_from_documents(documents_musk)

for idx, node in enumerate(base_nodes_musk):
    node.id_ = f"musk_node-{idx}"


### 为每个 BaseNode 生成摘要，并将摘要生成 IndexNode 并入 Node集合中

In [ ]:
node_to_metadata_musk = {}

extractor = SummaryExtractor(llm=llm_summarize, summaries=["self"], show_progress=True)
metadata_dicts = extractor.extract(base_nodes_musk)

for node, metadata in zip(base_nodes_musk, metadata_dicts):
    if node.node_id not in node_to_metadata_musk:
        node_to_metadata_musk[node.node_id] = metadata
    else:
        node_to_metadata_musk[node.node_id].update(metadata)


all_nodes_musk = copy.deepcopy(base_nodes_musk)
for node_id, metadata in node_to_metadata_musk.items():
    for val in metadata.values():
        all_nodes_musk.append(IndexNode(text=val, index_id=node_id))

all_nodes_dict_musk = {n.node_id: n for n in all_nodes_musk}

### 通过 VectorStoreIndex 将 Elon_Musk Wiki 的所有节点入向量数据库

In [ ]:
vector_index_chunk_musk = VectorStoreIndex(all_nodes_musk)

In [ ]:
vector_retriever_musk = vector_index_chunk_musk.as_retriever(similarity_top_k=5)

retriever_chunk_musk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_musk},
    node_dict=all_nodes_dict_musk,
    verbose=True,
)

### 创建 BM25 Retriever来补充 Vector 分块较大的细节缺失，并通过 QueryFusionRetriever 来倒数重排提升结果质量

In [ ]:
bm25_retriever_musk = BM25Retriever.from_defaults(docstore=vector_index_chunk_musk.docstore, similarity_top_k=5)


hybrid_retriever_musk = QueryFusionRetriever(
    [retriever_chunk_musk, bm25_retriever_musk],
    llm=llm,
    similarity_top_k=5,
    num_queries=4,  # set this to 1 to disable query generation
    mode="relative_score",
    use_async=True,
    verbose=True
)

### 测试 Elon Musk Wiki Retriever

In [ ]:
nodes = hybrid_retriever_musk.retrieve(
    "Tell me about the childhood of a billionaire who started at company at he age of 16"
)
for node in nodes:
    print(node.node.get_content())


Generated queries:
Here are three search queries related to the input query:
1. "Biographies of self-made billionaires who started their business as teenagers"
2. "Successful entrepreneurs who founded companies in their teenage years"
3. "Rags-to-riches stories of billionaires who started their journey at 16"
Let me know if you'd like me to generate more!
Retrieving with query id None: Tell me about the childhood of a billionaire who started at company at he age of 16


Retrieving text node: Securities and Exchange Commission (SEC) sued Musk, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired Twitter for $44 billion. He subsequently merged the company into newly created X Corp. and rebranded the service as X the following year. In March 2023, Musk founded xAI, an artificial intelligence company.
Musk has expressed views that have made him a polarizing figure. He has been criticized for making unscientific and misleading statements, including COVID-19 misinformation and antisemitic conspiracy theories. His ownership of Twitter has been similarly controversial, being marked by layoffs of large numbers of employees, an increase in hate speech and misinformation and disinformation on the website, and changes to Twitter Blue verification.


== Early life and education ==


=== Childhood and family

Retrieved node with id, entering: musk_node-5
Retrieving with query id musk_node-5: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-13
Retrieving with query id musk_node-13: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-3
Retrieving with query id musk_node-3: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-1
Retrieving with query id musk_node-1: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-19
Retrieving with query id musk_node-19: Here are three search queries related to the input query:
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...
Retrieving with query id None: 1. "Biographies of self-made billio

Retrieved node with id, entering: musk_node-0
Retrieving with query id musk_node-0: 1. "Biographies of self-made billionaires who started their business as teenagers"
Retrieved node with id, entering: musk_node-5
Retrieving with query id musk_node-5: 1. "Biographies of self-made billionaires who started their business as teenagers"
Retrieved node with id, entering: musk_node-3
Retrieving with query id musk_node-3: 1. "Biographies of self-made billionaires who started their business as teenagers"
Retrieved node with id, entering: musk_node-1
Retrieving with query id musk_node-1: 1. "Biographies of self-made billionaires who started their business as teenagers"
Retrieved node with id, entering: musk_node-4
Retrieving with query id musk_node-4: 1. "Biographies of self-made billionaires who started their business as teenagers"
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: musk_node-3
Retrieving with query id musk_node-3: 2. "Successful entrepreneurs who founded companies in their teenage years"
Retrieving text node: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. 
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the Unive

## 先用两篇 Wiki 做 Summery，然后分别索引上面各自的 Retriever

### 生成摘要，构建 IndexNode

In [ ]:
# 用摘要做入口索引
top_nodes = []
vector_retrievers = {}

for wiki_title in wiki_titles:
    # use LLM-generated summary
    summary_index = SummaryIndex.from_documents([docs_dict[wiki_title]])
    summarizer = summary_index.as_query_engine(response_mode="tree_summarize", llm=llm_summarize)
    response = await summarizer.aquery(
        f"Give me a summary of {wiki_title}"
    )

    wiki_summary = response.response

    print(f"**Summary for {wiki_title}: \n{wiki_summary}")
    wiki_node = IndexNode(text=wiki_summary, index_id=wiki_title)
    top_nodes.append(wiki_node)

**Summary for Michael Jordan: Michael Jordan is a renowned American former professional basketball player, entrepreneur, and sports icon. He is widely regarded as one of the greatest basketball players of all time. Born on February 17, 1963, Jordan played in the National Basketball Association (NBA) for the Chicago Bulls and Washington Wizards. He won six NBA championships, five Most Valuable Player (MVP) awards, and is the all-time leader in points per game with an average of 30.12.


**Summary for Elon Musk: Elon Musk is a renowned entrepreneur, business magnate, and engineer who has founded or co-founded several successful companies, including PayPal, SpaceX, Tesla, Neuralink, and The Boring Company. He is known for his innovative ideas, ambitious projects, and unconventional leadership style. Musk has been involved in various ventures, including electric cars, space exploration, renewable energy, and artificial intelligence. He has also been recognized for his entrepreneurial spirit and has been named one of the most influential people in the world by Time magazine.


### 两个索引节点构建入口的 Retriever

In [ ]:
top_vector_index = VectorStoreIndex(top_nodes, transformations=[node_parser])
top_vector_retriever = top_vector_index.as_retriever(similarity_top_k=1)

### 关于 Michael Jordan 和 Elon Musk 的相关问题都会索引到各自的 Retriever 去进一步执行

In [ ]:
top_recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": top_vector_retriever, "Michael Jordan":recursive_retriever_jordan, "Elon Musk":hybrid_retriever_musk},
    node_dict={**all_nodes_dict_jordan, **all_nodes_dict_musk},
    verbose=True
)

In [ ]:
nodes = top_recursive_retriever.retrieve(
    "Tell me about the childhood of a billionaire who started at company at he age of 16"
)
for node in nodes:
    print(node.node.get_content())

## 测试
告诉我一位16岁开始在公司工作的亿万富翁的童年

In [ ]:
response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    top_recursive_retriever,
    response_synthesizer=response_synthesizer,
    verbose=True
)

response = query_engine.query("Tell me about the childhood of a billionaire who started at company at the age of 16")
display_response(response)

Retrieving with query id None: Tell me about the childhood of a billionaire who started at company at the age of 16
Retrieved node with id, entering: Elon Musk
Retrieving with query id Elon Musk: Tell me about the childhood of a billionaire who started at company at the age of 16


Generated queries:
Here are three search queries related to the input query:
Richard Branson's early life and entrepreneurial ventures as a teenager
Biographies of self-made billionaires who started their business empires as minors
Successful entrepreneurs who dropped out of school to pursue their startup dreams
Retrieving with query id None: Tell me about the childhood of a billionaire who started at company at the age of 16
Retrieving text node: Securities and Exchange Commission (SEC) sued Musk, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired Twitter for $44 billion. He subsequently merged the company into newly created X Corp. and rebranded the service as X the following year. In March 2023, Musk founded xAI, an artificial intelligence company.
Musk has expressed views that have made him a polarizing figure. He has been

Retrieved node with id, entering: musk_node-5
Retrieving with query id musk_node-5: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-13
Retrieving with query id musk_node-13: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-3
Retrieving with query id musk_node-3: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-1
Retrieving with query id musk_node-1: Here are three search queries related to the input query:
Retrieved node with id, entering: musk_node-19
Retrieving with query id musk_node-19: Here are three search queries related to the input query:
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...
BM25Retriever does not support embeddings, skipping...
Retrieving with query id None: Richard Branson's early life and en

Retrieved node with id, entering: musk_node-0
Retrieving with query id musk_node-0: Richard Branson's early life and entrepreneurial ventures as a teenager
Retrieved node with id, entering: musk_node-3
Retrieving with query id musk_node-3: Richard Branson's early life and entrepreneurial ventures as a teenager
Retrieved node with id, entering: musk_node-1
Retrieving with query id musk_node-1: Richard Branson's early life and entrepreneurial ventures as a teenager
Retrieving text node: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. 
A member of the wealthy S

Retrieving text node: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. 
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped out a

**`Final Response:`** Based on the provided context, the billionaire who started a company at the age of 16 is Elon Musk. Elon Musk was born on June 28, 1971, in Pretoria, South Africa. His mother, Maye Musk, is a model and dietitian, and his father, Errol Musk, is a South African electromechanical engineer. Elon has a younger brother, Kimbal, and a younger sister, Tosca. The family was wealthy during Elon's youth. After his parents divorced in 1980, Elon chose to live primarily with his father. Elon later regretted his decision and became estranged from his father. At age 12, Elon developed an interest in computing and video games, teaching himself how to program from the VIC-20 user manual. At age 12, Elon sold his BASIC-based game Blastar to PC and Office Technology magazine for approximately $500.